In [1]:
#------------------------------------------------------------------------
# WES 207 SBUS Demo Code
# 
#-------------------------------------------------------------------------

from pynq import Overlay
from pynq import MMIO
import time
import numpy as np

print('')
print('done')


done


In [15]:
# Load bitstream
#ol = Overlay('SBUS_DEMO_100M_tx_ext_clks_test5.bit')
ol = Overlay('FLIGHT_MAIN_PID_TEST_4.bit')

# Download bitstream to FPGA
t_before_bitstream = time.time()
ol.download()
t_after_bitstream = time.time()
print('  ')
print(t_after_bitstream - t_before_bitstream, 'seconds to program bitstream')

print('')
print('done')

  
0.5412580966949463 seconds to program bitstream

done


In [16]:
# Address space definitions

UART_16550_BASE_ADDRESS         = 0x43C00000
UART_DRIVER_BASE_ADDRESS        = 0x43C10000
#UART_DATA_BASE_ADDRESS         = 0x40004000
RC_RECEIVER_BASE_ADDRESS        = 0x40000000
RC_RECEIVER_DATA_BASE_ADDRESS   = 0x40000020  # 0x20 for raw SBUS data
RC_RECEIVER_TEST_BASE_ADDRESS   = 0x40004000
FLIGHT_MAIN_CTRL_BASE_ADDRESS   = 0x40010000
FLIGHT_MAIN_CMD_BASE_ADDRESS    = 0x40020010  # 0x10 for scaled SBUS data
FLIGHT_MAIN_TEST_BASE_ADDRESS   = 0x40064000
PID_CTRL_BASE_ADDRESS           = 0x40030000
PID_KP_BASE_ADDRESS             = 0x40030020
PID_KI_BASE_ADDRESS             = 0x40030050
PID_KD_BASE_ADDRESS             = 0x40030040
PID_INPUT_BASE_ADDRESS          = 0x40040010  # 0x10 CMD, 0x20 meas
PID_TEST_BASE_ADDRESS           = 0x40054000



ADDRESS_RANGE1 = 0x1000   # 4k
ADDRESS_RANGE2 = 0x4000   # 16k
ADDRESS_RANGE3 = 0x10000  # 64k



print('Address Definitions complete')
print('')
print('done')

Address Definitions complete

done


In [17]:
# setting PID gains

index_0 = 0x00
index_1 = 0x04
index_2 = 0x08
index_3 = 0x0C
index_4 = 0x10
index_5 = 0x14

kp_gains = MMIO(PID_KP_BASE_ADDRESS,ADDRESS_RANGE1, True)
ki_gains = MMIO(PID_KI_BASE_ADDRESS,ADDRESS_RANGE1, True)
kd_gains = MMIO(PID_KD_BASE_ADDRESS,ADDRESS_RANGE1, True)

kp_gains.write(index_0,0x01)
kp_gains.write(index_1,0x01)
kp_gains.write(index_2,0x01)
kp_gains.write(index_3,0x01)
kp_gains.write(index_4,0x01)
kp_gains.write(index_5,0x01)

ki_gains.write(index_0,0x00)
ki_gains.write(index_1,0x00)
ki_gains.write(index_2,0x00)
ki_gains.write(index_3,0x00)

kd_gains.write(index_0,0x00)
kd_gains.write(index_1,0x00)
kd_gains.write(index_2,0x00)
kd_gains.write(index_3,0x00)


test = kp_gains.read(index_0)
print(test)
test = kp_gains.read(index_1)
print(test)
test = kp_gains.read(index_2)
print(test)
test = kp_gains.read(index_3)
print(test)
test = kp_gains.read(index_4)
print(test)
test = kp_gains.read(index_5)
print(test)

test = ki_gains.read(index_0)
print(test)
test = ki_gains.read(index_1)
print(test)
test = ki_gains.read(index_2)
print(test)
test = ki_gains.read(index_3)
print(test)

test = kd_gains.read(index_0)
print(test)
test = kd_gains.read(index_1)
print(test)
test = kd_gains.read(index_2)
print(test)
test = kd_gains.read(index_3)
print(test)




print('')
print('done')

MMIO Debug: MMIO(address, size) = (40030020, 1000 bytes).
MMIO Debug: MMIO(address, size) = (40030050, 1000 bytes).
MMIO Debug: MMIO(address, size) = (40030040, 1000 bytes).
MMIO Debug: Writing 4 bytes to offset 0: 1
MMIO Debug: Writing 4 bytes to offset 4: 1
MMIO Debug: Writing 4 bytes to offset 8: 1
MMIO Debug: Writing 4 bytes to offset c: 1
MMIO Debug: Writing 4 bytes to offset 10: 1
MMIO Debug: Writing 4 bytes to offset 14: 1
MMIO Debug: Writing 4 bytes to offset 0: 0
MMIO Debug: Writing 4 bytes to offset 4: 0
MMIO Debug: Writing 4 bytes to offset 8: 0
MMIO Debug: Writing 4 bytes to offset c: 0
MMIO Debug: Writing 4 bytes to offset 0: 0
MMIO Debug: Writing 4 bytes to offset 4: 0
MMIO Debug: Writing 4 bytes to offset 8: 0
MMIO Debug: Writing 4 bytes to offset c: 0
MMIO Debug: Reading 4 bytes from offset 0
1
MMIO Debug: Reading 4 bytes from offset 4
1
MMIO Debug: Reading 4 bytes from offset 8
1
MMIO Debug: Reading 4 bytes from offset c
1
MMIO Debug: Reading 4 bytes from offset 10
1
M

In [18]:
# configuring UART16550 core

UART_core = MMIO(UART_16550_BASE_ADDRESS,ADDRESS_RANGE3, True)   # 64k
UART_Driver = MMIO(UART_DRIVER_BASE_ADDRESS,ADDRESS_RANGE1, True)

RC_Driver = MMIO(RC_RECEIVER_BASE_ADDRESS,ADDRESS_RANGE1, True)
RC_Input_Data = MMIO(RC_RECEIVER_DATA_BASE_ADDRESS,ADDRESS_RANGE1, True)
RC_TEST_Data = MMIO(RC_RECEIVER_TEST_BASE_ADDRESS,ADDRESS_RANGE2, True)

Flight_Main_core = MMIO(FLIGHT_MAIN_CTRL_BASE_ADDRESS,ADDRESS_RANGE1, True)
Flight_Main_Input_Data = MMIO(FLIGHT_MAIN_CMD_BASE_ADDRESS,ADDRESS_RANGE1, True)
Flight_Main_TEST_Data = MMIO(FLIGHT_MAIN_TEST_BASE_ADDRESS,ADDRESS_RANGE2, True)

PID_core = MMIO(PID_CTRL_BASE_ADDRESS,ADDRESS_RANGE1, True)
PID_Input_Data = MMIO(PID_INPUT_BASE_ADDRESS,ADDRESS_RANGE1, True)
PID_Test_Data = MMIO(PID_TEST_BASE_ADDRESS,ADDRESS_RANGE2, True)   # 16k


SCRATCH_REG      =   0x101C 

# setting WHO_AM_I value
#UART_core.write(SCRATCH_REG,0x45)




print('UART16550 Core Configuration Complete')
print('')
print('done')

MMIO Debug: MMIO(address, size) = (43c00000, 10000 bytes).
MMIO Debug: MMIO(address, size) = (43c10000, 1000 bytes).
MMIO Debug: MMIO(address, size) = (40000000, 1000 bytes).
MMIO Debug: MMIO(address, size) = (40000020, 1000 bytes).
MMIO Debug: MMIO(address, size) = (40004000, 4000 bytes).
MMIO Debug: MMIO(address, size) = (40010000, 1000 bytes).
MMIO Debug: MMIO(address, size) = (40020010, 1000 bytes).
MMIO Debug: MMIO(address, size) = (40064000, 4000 bytes).
MMIO Debug: MMIO(address, size) = (40030000, 1000 bytes).
MMIO Debug: MMIO(address, size) = (40040010, 1000 bytes).
MMIO Debug: MMIO(address, size) = (40054000, 4000 bytes).
UART16550 Core Configuration Complete

done


In [19]:
# NOTE, starting last core in chain first

# setting ap_start/auto reset on UART Driver
#PID_core.write(0x00,0x01)  # runs once
PID_core.write(0x00,0x81) # runs continuously

# setting ap_start/auto reset on UART Driver
#Flight_Main_core.write(0x00,0x01)  # runs once
Flight_Main_core.write(0x00,0x81) # runs continuously

# setting ap_start/auto reset on RC Receiver
#RC_Driver.write(0x00,0x01)  # runs once
RC_Driver.write(0x00,0x81) # runs continuously

# setting ap_start/auto reset on UART Driver
#UART_Driver.write(0x00,0x01)  # runs once
UART_Driver.write(0x00,0x81) # runs continuously




# reading back CONFIG registers for HLS cores
CONFIG_REG1 = UART_Driver.read(0x00)
CONFIG_REG2 = RC_Driver.read(0x00)
CONFIG_REG3 = Flight_Main_core.read(0x00)
CONFIG_REG4 = PID_core.read(0x00)

print('CONFIG_REG UART: ')
print(CONFIG_REG1)
print('')
print('CONFIG_REG RC: ')
print(CONFIG_REG2)
print('')
print('CONFIG_REG Flight Main: ')
print(CONFIG_REG3)
print('')
print('CONFIG_REG PID: ')
print(CONFIG_REG4)
print('')



print('')
print('')

MMIO Debug: Writing 4 bytes to offset 0: 81
MMIO Debug: Writing 4 bytes to offset 0: 81
MMIO Debug: Writing 4 bytes to offset 0: 81
MMIO Debug: Writing 4 bytes to offset 0: 81
MMIO Debug: Reading 4 bytes from offset 0
MMIO Debug: Reading 4 bytes from offset 0
MMIO Debug: Reading 4 bytes from offset 0
MMIO Debug: Reading 4 bytes from offset 0
CONFIG_REG UART: 
131

CONFIG_REG RC: 
131

CONFIG_REG Flight Main: 
131

CONFIG_REG PID: 
131





In [21]:
# testing

LINE_STATUS_REG  =   0x1014 

test_read = UART_core.read(SCRATCH_REG)

print('WHO_AM_I: ' + str(test_read))


# reading FIFO status

OVERRUN_ERROR = UART_core.read(LINE_STATUS_REG)
OVERRUN_ERROR = OVERRUN_ERROR & 0x02 # grabbing bit 1


print('OVERRUN_ERROR: ')
print(OVERRUN_ERROR)
print('')


print('')
print('done')

MMIO Debug: Reading 4 bytes from offset 101c
WHO_AM_I: 69
MMIO Debug: Reading 4 bytes from offset 1014
OVERRUN_ERROR: 
0


done


In [63]:
# testing reads from UART Driver and RC Reciever Data



index_0 = 0x00
index_1 = 0x04
index_2 = 0x08
index_3 = 0x0C
index_4 = 0x10
index_5 = 0x14
index_6 = 0x18
index_7 = 0x1C 
index_8 = 0x20
index_9 = 0x24
index_10 = 0x28
index_11 = 0x2C
index_12 = 0x30
index_13 = 0x34
index_14 = 0x38
index_15 = 0x3C 
index_16 = 0x40
index_17 = 0x44
index_18 = 0x48
index_19 = 0x4C
index_20 = 0x50
index_21 = 0x54
index_22 = 0x58
index_23 = 0x5C 
index_24 = 0x60
index_25 = 0x64
index_26 = 0x68
index_27 = 0x6C
index_28 = 0x70
index_29 = 0x74

# HLS test read

test1 = RC_TEST_Data.read(index_0)
test2 = RC_TEST_Data.read(index_1)
test3 = RC_TEST_Data.read(index_2)
test4 = RC_TEST_Data.read(index_3)
test5 = RC_TEST_Data.read(index_4)
test6 = RC_TEST_Data.read(index_5)

test7 = RC_TEST_Data.read(index_6)
test8 = RC_TEST_Data.read(index_7)
test9 = RC_TEST_Data.read(index_8)
test10 = RC_TEST_Data.read(index_9)
test11 = RC_TEST_Data.read(index_10)
test12 = RC_TEST_Data.read(index_11)

print('Throttle Input RC Core: ')
print("%4d"% (test1 >> 16) )
print('')
print('Roll Input RC Core: ')
print("%4d"% (test2 >> 16))
print('')
print('Pitch Input RC Core: ')
print("%4d"% (test3 >> 16))
print('')
print('Yaw Input RC Core: ')
print("%4d"% (test4 >> 16))
print('')
print('Arm Input RC Core: ')
print("%1d"% (test5 >> 16))
print('')
print('Mode Input RC Core: ')
print("%1d"% (test6 >> 16))
print('')
print('Throttle Output RC Core: ')
print("%5d"% (test7))
print('')
print('Roll Output RC Core: ')
print("%5d"% (test8))
print('')
print('Pitch Output RC Core: ')
print("%5d"% (test9))
print('')
print('Yaw Output RC Core: ')
print("%5d"% (test10 << 8))
print('')
print('Arm Output RC Core: ')
print("%1d"% (test11 << 8))
print('')
print('Mode Output RC Core: ')
print("%1d"% (test12 << 8))
print('')


MMIO Debug: Reading 4 bytes from offset 0
MMIO Debug: Reading 4 bytes from offset 4
MMIO Debug: Reading 4 bytes from offset 8
MMIO Debug: Reading 4 bytes from offset c
MMIO Debug: Reading 4 bytes from offset 10
MMIO Debug: Reading 4 bytes from offset 14
MMIO Debug: Reading 4 bytes from offset 18
MMIO Debug: Reading 4 bytes from offset 1c
MMIO Debug: Reading 4 bytes from offset 20
MMIO Debug: Reading 4 bytes from offset 24
MMIO Debug: Reading 4 bytes from offset 28
MMIO Debug: Reading 4 bytes from offset 2c
Throttle Input RC Core: 
 172

Roll Input RC Core: 
 997

Pitch Input RC Core: 
1003

Yaw Input RC Core: 
1000

Arm Input RC Core: 
1811

Mode Input RC Core: 
172

Throttle Output RC Core: 
    0

Roll Output RC Core: 
32608

Pitch Output RC Core: 
32848

Yaw Output RC Core: 
8378368

Arm Output RC Core: 
16777216

Mode Output RC Core: 
0



In [66]:
# testing HLS core reads 


test1 = Flight_Main_TEST_Data.read(index_0)
test2 = Flight_Main_TEST_Data.read(index_1)
test3 = Flight_Main_TEST_Data.read(index_2)
test4 = Flight_Main_TEST_Data.read(index_3)
test5 = Flight_Main_TEST_Data.read(index_4)
test6 = Flight_Main_TEST_Data.read(index_5)

test7 = Flight_Main_TEST_Data.read(index_6)
test8 = Flight_Main_TEST_Data.read(index_7)
test9 = Flight_Main_TEST_Data.read(index_8)
test10 = Flight_Main_TEST_Data.read(index_9)
test11 = Flight_Main_TEST_Data.read(index_10)
test12 = Flight_Main_TEST_Data.read(index_11)

print('Throttle Input Flight Main: ')
print("%5d"% (test1 << 16))
print('')
print('Roll Input Flight Main: ')
print("%5d"% (test2))
print('')
print('Pitch Input Flight Main: ')
print("%5d"% (test3 >> 16))
print('')
print('Yaw Input Flight Main: ')
print("%5d"% (test4 >> 16))
print('')
print('Arm Input Flight Main: ')
print("%1d"% (test5))
print('')
print('Mode Input Flight Main: ')
print("%1d"% (test6))
print('')
print('Throttle Output Flight Main: ')
print("%5.10d"% (test7 >> 16))
print('')
print('Roll Output Flight Main: ')
print("%5.10d"% (test8 >> 16))
print('')
print('Pitch Output Flight Main: ')
print("%5.10d"% (test9 >> 16))
print('')
print('Yaw Output Flight Main: ')
print("%5.10d"% (test10 >> 16))
print('')
print('Arm Output Flight Main: ')
print("%1d"% (test11))
print('')
print('Mode Output Flight Main: ')
print("%1d"% (test12))
print('')


TEST_READ1  = PID_Test_Data.read(index_0)
TEST_READ2  = PID_Test_Data.read(index_1)
TEST_READ3  = PID_Test_Data.read(index_2)
TEST_READ4  = PID_Test_Data.read(index_3)
TEST_READ5  = PID_Test_Data.read(index_4)
TEST_READ6  = PID_Test_Data.read(index_5)
TEST_READ7  = PID_Test_Data.read(index_6)
TEST_READ8  = PID_Test_Data.read(index_7)
TEST_READ9  = PID_Test_Data.read(index_8)
TEST_READ10 = PID_Test_Data.read(index_9)
TEST_READ11 = PID_Test_Data.read(index_10)
TEST_READ12 = PID_Test_Data.read(index_11)
TEST_READ13 = PID_Test_Data.read(index_12)
TEST_READ14 = PID_Test_Data.read(index_13)





print('Throttle Input PID: ')
print("%5d"% (TEST_READ1 << 8))
print('')
print('Roll Input PID: ')
print(TEST_READ2)
print('')
print('Pitch Input PID: ')
print(TEST_READ3 >> 16)
print('')
print('Yaw Input PID: ')
print(TEST_READ4 >> 16)
print('')
print('ARM Flag PID: ')
print(TEST_READ5)
print('')
print('Flight Mode Flag PID: ')
print(TEST_READ6)
print('')
print('M1 out PID: ')
print(TEST_READ7 >> 16)
print('')
print('M2 out PID: ')
print(TEST_READ8 >> 16)
print('')
print('M3 out PID: ')
print(TEST_READ9 >> 16)
print('')
print('M4 out PID: ')
print(TEST_READ10 >> 16)
print('')
print('M5 out PID: ')
print(TEST_READ11 >> 16)
print('')
print('M6 out PID: ')
print(TEST_READ12 >> 16)
print('')
print('M7 out PID: ')
print(TEST_READ13 >> 16)
print('')
print('M8 out PID: ')
print(TEST_READ14 >> 16)
print('')





print('')
print('done')

MMIO Debug: Reading 4 bytes from offset 0
MMIO Debug: Reading 4 bytes from offset 4
MMIO Debug: Reading 4 bytes from offset 8
MMIO Debug: Reading 4 bytes from offset c
MMIO Debug: Reading 4 bytes from offset 10
MMIO Debug: Reading 4 bytes from offset 14
MMIO Debug: Reading 4 bytes from offset 18
MMIO Debug: Reading 4 bytes from offset 1c
MMIO Debug: Reading 4 bytes from offset 20
MMIO Debug: Reading 4 bytes from offset 24
MMIO Debug: Reading 4 bytes from offset 28
MMIO Debug: Reading 4 bytes from offset 2c
Throttle Input Flight Main: 
    0

Roll Input Flight Main: 
    0

Pitch Input Flight Main: 
    0

Yaw Input Flight Main: 
    0

Arm Input Flight Main: 
1

Mode Input Flight Main: 
0

Throttle Output Flight Main: 
0000000000

Roll Output Flight Main: 
0000000000

Pitch Output Flight Main: 
0000000000

Yaw Output Flight Main: 
0000000000

Arm Output Flight Main: 
1

Mode Output Flight Main: 
0

MMIO Debug: Reading 4 bytes from offset 0
MMIO Debug: Reading 4 bytes from offset 4
MMIO

In [ ]:

# HLS test write
#UART_Data.write(0x00,0x45)
RC_Norm_Data.write(0x00,0x45)
RC_Rev_Data.write(0x00,0x45)
RC_Chan_Data.write(0x00,0x45)

# HLS test read
#TEST_READ = UART_Data.read(0x00)
TEST_READ1 = RC_Norm_Data.read(0x00)
TEST_READ2 = RC_Rev_Data.read(0x00)
TEST_READ3 = RC_Chan_Data.read(0x00)

print('Test Data: ')
print(TEST_READ1)
print('')
print('Test Data: ')
print(TEST_READ2)
print('')
print('Test Data: ')
print(TEST_READ3)
print('')

print('')
print('done')

In [ ]:
# allocating storage array for data

ARRAY_SIZE = 50000

buf_data = np.empty([ARRAY_SIZE,], dtype=np.int8)

print('Array allocation complete')
print('')
print('done')

In [ ]:
# test code

# writing to TX register
#UART_core.write(TX_HOLD_REG,0x55)
#UART_core.write(TX_HOLD_REG,0xad)
#UART_core.write(TX_HOLD_REG,0xbe)
#UART_core.write(TX_HOLD_REG,0xef)

print('transmit complete')
print('')
print('done')

In [ ]:
# grabbing data from the UART bus

NUM_BYTES = 25

BIT_ONE = 0x00000001
buffer = [0]*(NUM_BYTES)
BYTES_READ = 0


# resetting RX FIFO
#UART_core.write(FIFO_CNTRL_REG, 0x7)

while(BYTES_READ != (NUM_BYTES)):
    
    #read from the RX FIFO when data present
    while((UART_core.read(LINE_STATUS_REG) & 0x1) != 1):
        pass
    
    buffer[0] = UART_core.read(RX_BUF_REG)
    if ( buffer[0] == START_BYTE):
    #if ( buffer[0]):
        BYTES_READ += 1
        for i in range(NUM_BYTES - 1):
            while((UART_core.read(LINE_STATUS_REG) & 0x1) != 1):
                pass
            buffer[BYTES_READ] = UART_core.read(RX_BUF_REG)
            BYTES_READ += 1
        #if( buffer[24] != STOP_BYTE ):
            #BYTES_READ = 0
    else:
        pass
    #print('Total bytes read: ', BYTES_READ)

    

    
#print(buffer)  # printing buffer contents in decimal
print('')
print(''.join('%02x'%i for i in buffer))   # printing buffer contents in hex

print('')
print('done')

In [ ]:
# testing

WHO_AM_I = UART_core.read(SCRATCH_REG)

print('WHO_AM_I: ' + str(WHO_AM_I))

print('')
print('done')

In [ ]:
# Write OFDM Receiver results to output file for easier comparison
with open('output_compare.dat', 'w') as file_out:
    file_out.write("-----------------------------------------------------------------\n")
    file_out.write("             Golden Output versus OFDM Receiver Output           \n")
    file_out.write("-----------------------------------------------------------------\n\n")
    for k in range(FSIZE):
        file_out.write("index "+str(k)+",\t R: "+str(inp_R[k])+",\t I: "+str(inp_I[k])+",\t Gold: "+str(golden_D[k])+",\t OFDM: "+str(out_D[k])+"\n")

In [ ]:
# parsing buffered packet into channel data

# channel 1   Throttle
# channel 2   Roll
# channel 3   Pitch
# channel 4   yaw

NUM_CHANNELS = 4

channel = [0]*NUM_CHANNELS

# bit masking to parse 11 bit channel data from buffer
channel[0] = (buffer[1]      | buffer[2] << 8)                    & 0x7FF
channel[1] = (buffer[2] >> 3 | buffer[3] << 5)                    & 0x7FF
channel[2] = (buffer[3] >> 6 | buffer[4] << 2 | buffer[5] << 10)  & 0x7FF
channel[3] = (buffer[5] >> 1 | buffer[6] << 7)                    & 0x7FF


print('Throttle Command:  ', channel[0])
print('Roll Command:      ', channel[1])
print('Pitch Command:     ', channel[2])
print('Yaw Command:       ', channel[3])

print('')
print('done')

In [ ]:
#########################################
# old code
#########################################

# grabbing data from the UART bus

BIT_ONE = 0x00000001
buffer = [0]*(NUM_BYTES)
#buffer = [0]*(NUM_BYTES-1)
BYTES_READ = 0

test = [0]*(NUM_BYTES)


while(BYTES_READ != (NUM_BYTES)):
#while(BYTES_READ != (NUM_BYTES - 1)):
    
    #read from the RX FIFO when data present
    #while((mmio.read(LINE_STATUS_REG) & BIT_ONE) != 1):
        #pass
    
    #print(mmio.read(RX_BUF_REG))
    #print(idx)
    
    #if(mmio.read(RX_BUF_REG) != 240):
        #pass
        #print('hard pass')
        
        
    for index in range(NUM_BYTES):
        buffer[index] = mmio.read(RX_BUF_REG)
        BYTES_READ += 1
        
    #temp = mmio.read(RX_BUF_REG)
    #if (temp != 240):
    #    print(temp)
    #else:
    #    buffer[BYTES_READ] = mmio.read(RX_BUF_REG)
    #    BYTES_READ += 1

print(buffer) 
print('')
print(''.join('%02x'%i for i in buffer))
print('')

for i in range(NUM_BYTES):
    test[i] = ~buffer[i]
    
print(test)
print('')
print(''.join('%02x'%i for i in test))

In [ ]:
# testing grabbing channel 1

print('buffer[1]:             ', buffer[1])
print('buffer[2]:             ', buffer[2])
print('buffer[2] last 3 bits: ', buffer[2] & 0x07)

temp = buffer[1] | (buffer[2] << 8)
temp2 = temp & 0x07FF
temp3 = (buffer[1] | buffer[2] << 8) & 0x07FF
 
print('temp:                  ', temp)
print('temp2:                 ', temp2)
print('temp3:                 ', temp3)

start = buffer[0]
print('start byte:            ', start)
channel_1 = (buffer[1] | ( buffer[2] << 8 )) & 0x07FF

print('channel 1:             ', channel_1)

In [ ]:
# TRAVIS IDEAS for infinite graph display

while !quit:
  get packet
  update the graph
  send stuff if the user wants to send stuff
    i.e. break commands